## Importing necessary libraries

In [1]:
import cv2
import time
import numpy as np
from random import randint
import argparse
import sys
import time
%run "C:/Users/birud/OneDrive - iitgn.ac.in/github/openpose_clone/python/import_pyopenpose.py"
print("pyopenpose imported")

pyopenpose imported


## Defining some variables

In [2]:
threshold = 0.2
#Body_25 model use 25 points
key_points = {
    0:  "Nose", 1:  "Neck", 2:  "RShoulder", 3:  "RElbow", 4:  "RWrist", 5:  "LShoulder", 6:  "LElbow",
    7:  "LWrist", 8:  "MidHip", 9:  "RHip", 10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee",
    14: "LAnkle", 15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe", 20: "LSmallToe",
    21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"
}

#Body_25 keypoint pairs 
POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],     #arm, shoulder line
            [1,8], [8,9], [9,10], [10,11], [8,12], [12,13], [13,14],  #2 leg
            [11,24], [11,22], [22,23], [14,21],[14,19],[19,20],    #2 foot  
            [1,0], [0,15], [15,17], [0,16], [16,18], #face
#             [2,17], [5,18]
            ]  

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

nPoints = 25

## Importing the openpose model from the local system

In [3]:
protoFile = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\openpose_clone\models\pose\body_25\pose_deploy.prototxt"
weightsFile = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\openpose_clone\models\pose\body_25\pose_iter_584000.caffemodel"

## Getting the frame rate of the video

In [4]:
i = input("Enter the video path: ")
args.video = r"%s" % i
cap = cv2.VideoCapture(args.video)
ret, img = cap.read()
if ret == False:
    print('Video File Read Error')
    sys.exit(0)
frameHeight, frameWidth, c = img.shape

# Get the frames per second (fps) of the video
fps = cap.get(cv2.CAP_PROP_FPS)

print(f"Frame rate of the video: {fps} frames per second")

Enter the video path: C:\Users\birud\OneDrive - iitgn.ac.in\github\openpose_clone\examples\media\video2.mp4
Frame rate of the video: 25.0 frames per second


In [5]:
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out_video_path = r'C:\Users\birud\OneDrive - iitgn.ac.in\github\Pose-estimation\files\processed video\{}-{}-output.mp4'.format(args.model, args.device)
out_video = cv2.VideoWriter(out_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (frameWidth, frameHeight))

## The Below Code applies the model for each frame

In [6]:
frame = 0
inHeight = 368
params = dict()
params["model_folder"] = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\openpose_clone\models"
params["net_resolution"] = "368x-1"
params["display"] = "0"
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()
t_elapsed = 0.0

# Set initial values for thickness
line_thickness = 2
circle_thickness = 2

while cap.isOpened():
    f_st = time.time()
    ret, img = cap.read()
    if ret == False:
        break
    frame += 1
    datum = op.Datum()
    datum.cvInputData = img
    opWrapper.emplaceAndPop(op.VectorDatum([datum]))

    # Check if poseKeypoints is not None before accessing its length
    if datum.poseKeypoints is not None:
        human_count = len(datum.poseKeypoints)
    else:
        human_count = 0

    frameClone = img.copy()
    for human in range(human_count):
        for j in range(nPoints):
            if datum.poseKeypoints[human][j][2] > threshold:
                circle_center = tuple(map(int, datum.poseKeypoints[human][j][0:2]))
                cv2.circle(frameClone, circle_center, circle_thickness, colors[j % 17], -1, cv2.LINE_AA)

    for human in range(human_count):
        for pair in POSE_PAIRS:
            if datum.poseKeypoints[human][pair[0]][2] <= threshold or datum.poseKeypoints[human][pair[1]][2] <= threshold:
                continue
            S = (int(datum.poseKeypoints[human][pair[0]][0]),
                 int(datum.poseKeypoints[human][pair[0]][1]))
            E = (int(datum.poseKeypoints[human][pair[1]][0]),
                 int(datum.poseKeypoints[human][pair[1]][1]))
            cv2.line(frameClone, S, E, colors[pair[0] % 17], line_thickness, cv2.LINE_AA)

    out_video.write(frameClone)
    f_elapsed = time.time() - f_st
    t_elapsed += f_elapsed
    print('Frame[%d] processed time[%4.2f]' % (frame, f_elapsed))

print('Total processed time[%4.2f]' % (t_elapsed))
print('avg frame processing rate :%4.2f' % (t_elapsed / frame))
cap.release()
out_video.release()

Frame[1] processed time[1.48]
Frame[2] processed time[0.07]
Frame[3] processed time[0.07]
Frame[4] processed time[0.08]
Frame[5] processed time[0.07]
Frame[6] processed time[0.07]
Frame[7] processed time[0.08]
Frame[8] processed time[0.07]
Frame[9] processed time[0.07]
Frame[10] processed time[0.07]
Frame[11] processed time[0.08]
Frame[12] processed time[0.07]
Frame[13] processed time[0.07]
Frame[14] processed time[0.08]
Frame[15] processed time[0.07]
Frame[16] processed time[0.08]
Frame[17] processed time[0.09]
Frame[18] processed time[0.08]
Frame[19] processed time[0.08]
Frame[20] processed time[0.08]
Frame[21] processed time[0.08]
Frame[22] processed time[0.08]
Frame[23] processed time[0.08]
Frame[24] processed time[0.07]
Frame[25] processed time[0.07]
Frame[26] processed time[0.09]
Frame[27] processed time[0.08]
Frame[28] processed time[0.08]
Frame[29] processed time[0.07]
Frame[30] processed time[0.07]
Frame[31] processed time[0.08]
Frame[32] processed time[0.07]
Frame[33] process

Frame[263] processed time[0.10]
Frame[264] processed time[0.12]
Frame[265] processed time[0.09]
Frame[266] processed time[0.08]
Frame[267] processed time[0.10]
Frame[268] processed time[0.10]
Frame[269] processed time[0.08]
Frame[270] processed time[0.10]
Frame[271] processed time[0.11]
Frame[272] processed time[0.09]
Frame[273] processed time[0.08]
Frame[274] processed time[0.08]
Frame[275] processed time[0.09]
Frame[276] processed time[0.10]
Frame[277] processed time[0.10]
Frame[278] processed time[0.08]
Frame[279] processed time[0.10]
Frame[280] processed time[0.10]
Frame[281] processed time[0.07]
Frame[282] processed time[0.10]
Frame[283] processed time[0.11]
Frame[284] processed time[0.08]
Frame[285] processed time[0.10]
Frame[286] processed time[0.10]
Frame[287] processed time[0.10]
Frame[288] processed time[0.10]
Frame[289] processed time[0.10]
Frame[290] processed time[0.08]
Frame[291] processed time[0.10]
Frame[292] processed time[0.11]
Frame[293] processed time[0.09]
Frame[29

Frame[522] processed time[0.11]
Frame[523] processed time[0.09]
Frame[524] processed time[0.09]
Frame[525] processed time[0.10]
Frame[526] processed time[0.11]
Frame[527] processed time[0.09]
Frame[528] processed time[0.10]
Frame[529] processed time[0.08]
Frame[530] processed time[0.10]
Frame[531] processed time[0.10]
Frame[532] processed time[0.09]
Frame[533] processed time[0.09]
Frame[534] processed time[0.10]
Frame[535] processed time[0.06]
Frame[536] processed time[0.10]
Frame[537] processed time[0.10]
Frame[538] processed time[0.10]
Frame[539] processed time[0.09]
Frame[540] processed time[0.10]
Frame[541] processed time[0.08]
Frame[542] processed time[0.09]
Frame[543] processed time[0.10]
Frame[544] processed time[0.10]
Frame[545] processed time[0.09]
Frame[546] processed time[0.10]
Frame[547] processed time[0.07]
Frame[548] processed time[0.09]
Frame[549] processed time[0.11]
Frame[550] processed time[0.08]
Frame[551] processed time[0.10]
Frame[552] processed time[0.10]
Frame[55

Frame[780] processed time[0.10]
Frame[781] processed time[0.08]
Frame[782] processed time[0.09]
Frame[783] processed time[0.10]
Frame[784] processed time[0.10]
Frame[785] processed time[0.09]
Frame[786] processed time[0.10]
Frame[787] processed time[0.09]
Frame[788] processed time[0.09]
Frame[789] processed time[0.10]
Frame[790] processed time[0.08]
Frame[791] processed time[0.09]
Frame[792] processed time[0.10]
Frame[793] processed time[0.09]
Frame[794] processed time[0.08]
Frame[795] processed time[0.09]
Frame[796] processed time[0.10]
Frame[797] processed time[0.08]
Frame[798] processed time[0.08]
Frame[799] processed time[0.09]
Frame[800] processed time[0.09]
Frame[801] processed time[0.10]
Frame[802] processed time[0.09]
Frame[803] processed time[0.08]
Frame[804] processed time[0.08]
Frame[805] processed time[0.08]
Frame[806] processed time[0.07]
Frame[807] processed time[0.08]
Frame[808] processed time[0.07]
Frame[809] processed time[0.08]
Frame[810] processed time[0.07]
Frame[81

Frame[1035] processed time[0.08]
Frame[1036] processed time[0.07]
Frame[1037] processed time[0.08]
Frame[1038] processed time[0.08]
Frame[1039] processed time[0.08]
Frame[1040] processed time[0.09]
Frame[1041] processed time[0.07]
Frame[1042] processed time[0.09]
Frame[1043] processed time[0.09]
Frame[1044] processed time[0.09]
Frame[1045] processed time[0.09]
Frame[1046] processed time[0.10]
Frame[1047] processed time[0.11]
Frame[1048] processed time[0.09]
Frame[1049] processed time[0.08]
Frame[1050] processed time[0.08]
Frame[1051] processed time[0.08]
Frame[1052] processed time[0.08]
Frame[1053] processed time[0.09]
Frame[1054] processed time[0.08]
Frame[1055] processed time[0.09]
Frame[1056] processed time[0.08]
Frame[1057] processed time[0.09]
Frame[1058] processed time[0.09]
Frame[1059] processed time[0.10]
Frame[1060] processed time[0.08]
Frame[1061] processed time[0.09]
Frame[1062] processed time[0.10]
Frame[1063] processed time[0.09]
Frame[1064] processed time[0.09]
Frame[1065

...

Frame[2789] processed time[0.10]
Frame[2790] processed time[0.08]
Frame[2791] processed time[0.10]
Frame[2792] processed time[0.10]
Frame[2793] processed time[0.09]
Frame[2794] processed time[0.09]
Frame[2795] processed time[0.10]
Frame[2796] processed time[0.09]
Frame[2797] processed time[0.09]
Frame[2798] processed time[0.10]
Frame[2799] processed time[0.10]
Frame[2800] processed time[0.09]
Frame[2801] processed time[0.09]
Frame[2802] processed time[0.08]
Frame[2803] processed time[0.08]
Frame[2804] processed time[0.08]
Frame[2805] processed time[0.08]
Frame[2806] processed time[0.10]
Frame[2807] processed time[0.10]
Frame[2808] processed time[0.10]
Frame[2809] processed time[0.09]
Frame[2810] processed time[0.08]
Frame[2811] processed time[0.08]
Frame[2812] processed time[0.08]
Frame[2813] processed time[0.08]
Frame[2814] processed time[0.07]
Frame[2815] processed time[0.07]
Frame[2816] processed time[0.10]
Frame[2817] processed time[0.09]
Frame[2818] processed time[0.10]
Frame[2819

Frame[3039] processed time[0.09]
Frame[3040] processed time[0.09]
Frame[3041] processed time[0.08]
Frame[3042] processed time[0.08]
Frame[3043] processed time[0.09]
Frame[3044] processed time[0.09]
Frame[3045] processed time[0.08]
Frame[3046] processed time[0.08]
Frame[3047] processed time[0.08]
Frame[3048] processed time[0.08]
Frame[3049] processed time[0.08]
Frame[3050] processed time[0.08]
Frame[3051] processed time[0.08]
Frame[3052] processed time[0.09]
Frame[3053] processed time[0.09]
Frame[3054] processed time[0.10]
Frame[3055] processed time[0.09]
Frame[3056] processed time[0.09]
Frame[3057] processed time[0.09]
Frame[3058] processed time[0.10]
Frame[3059] processed time[0.09]
Frame[3060] processed time[0.09]
Frame[3061] processed time[0.09]
Frame[3062] processed time[0.08]
Frame[3063] processed time[0.09]
Frame[3064] processed time[0.08]
Frame[3065] processed time[0.08]
Frame[3066] processed time[0.08]
Frame[3067] processed time[0.08]
Frame[3068] processed time[0.10]
Frame[3069

Frame[3290] processed time[0.08]
Frame[3291] processed time[0.09]
Frame[3292] processed time[0.09]
Frame[3293] processed time[0.09]
Frame[3294] processed time[0.09]
Frame[3295] processed time[0.09]
Frame[3296] processed time[0.11]
Frame[3297] processed time[0.09]
Frame[3298] processed time[0.09]
Frame[3299] processed time[0.09]
Frame[3300] processed time[0.09]
Frame[3301] processed time[0.09]
Frame[3302] processed time[0.08]
Frame[3303] processed time[0.08]
Frame[3304] processed time[0.08]
Frame[3305] processed time[0.08]
Frame[3306] processed time[0.08]
Frame[3307] processed time[0.08]
Frame[3308] processed time[0.08]
Frame[3309] processed time[0.09]
Frame[3310] processed time[0.09]
Frame[3311] processed time[0.08]
Frame[3312] processed time[0.08]
Frame[3313] processed time[0.09]
Frame[3314] processed time[0.07]
Frame[3315] processed time[0.08]
Frame[3316] processed time[0.10]
Frame[3317] processed time[0.09]
Frame[3318] processed time[0.10]
Frame[3319] processed time[0.09]
Frame[3320

Frame[3539] processed time[0.07]
Frame[3540] processed time[0.08]
Frame[3541] processed time[0.07]
Frame[3542] processed time[0.08]
Frame[3543] processed time[0.08]
Frame[3544] processed time[0.09]
Frame[3545] processed time[0.09]
Frame[3546] processed time[0.09]
Frame[3547] processed time[0.09]
Frame[3548] processed time[0.09]
Frame[3549] processed time[0.09]
Frame[3550] processed time[0.09]
Frame[3551] processed time[0.09]
Frame[3552] processed time[0.09]
Frame[3553] processed time[0.09]
Frame[3554] processed time[0.08]
Frame[3555] processed time[0.08]
Frame[3556] processed time[0.10]
Frame[3557] processed time[0.10]
Frame[3558] processed time[0.10]
Frame[3559] processed time[0.09]
Frame[3560] processed time[0.10]
Frame[3561] processed time[0.07]
Frame[3562] processed time[0.09]
Frame[3563] processed time[0.07]
Frame[3564] processed time[0.06]
Frame[3565] processed time[0.07]
Frame[3566] processed time[0.10]
Frame[3567] processed time[0.09]
Frame[3568] processed time[0.09]
Frame[3569

Frame[3788] processed time[0.09]
Frame[3789] processed time[0.06]
Frame[3790] processed time[0.08]
Frame[3791] processed time[0.07]
Frame[3792] processed time[0.10]
Frame[3793] processed time[0.08]
Frame[3794] processed time[0.08]
Frame[3795] processed time[0.08]
Frame[3796] processed time[0.08]
Frame[3797] processed time[0.06]
Frame[3798] processed time[0.09]
Frame[3799] processed time[0.09]
Frame[3800] processed time[0.08]
Frame[3801] processed time[0.07]
Frame[3802] processed time[0.07]
Frame[3803] processed time[0.08]
Frame[3804] processed time[0.10]
Frame[3805] processed time[0.10]
Frame[3806] processed time[0.09]
Frame[3807] processed time[0.08]
Frame[3808] processed time[0.08]
Frame[3809] processed time[0.07]
Frame[3810] processed time[0.09]
Frame[3811] processed time[0.10]
Frame[3812] processed time[0.10]
Frame[3813] processed time[0.09]
Frame[3814] processed time[0.09]
Frame[3815] processed time[0.08]
Frame[3816] processed time[0.07]
Frame[3817] processed time[0.08]
Frame[3818